In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.utils as vutils
from torch.optim import SGD
import torch.utils.data as Data
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 加载数据

In [2]:
train_data=torchvision.datasets.MNIST(
    root='./data/MNIST',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=False)
train_loader=Data.DataLoader(
    dataset=train_data,
    batch_size=128,
    shuffle=True,
    num_workers=2)

for step,(batch_X,batch_y)in enumerate(train_loader):
    if step>0:break

print(batch_X.shape)
print(batch_y.shape)

torch.Size([128, 1, 28, 28])
torch.Size([128])


In [3]:
test_data=torchvision.datasets.MNIST(
    root='./data/MNIST',
    train=False,
    download=False)
test_data_X=test_data.data.type(torch.FloatTensor)/255.0
test_data_X=torch.unsqueeze(test_data_X,dim=1)
test_data_y=test_data.targets
print('test X shape:',test_data_X.shape)
print('test y shape:',test_data_y.shape)

test X shape: torch.Size([10000, 1, 28, 28])
test y shape: torch.Size([10000])


# 定义网络

In [2]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2,stride=2))
        self.conv2=nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc=nn.Sequential(
            nn.Linear(in_features=32*7*7,out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128,out_features=64),
            nn.ReLU()
        )
        self.out=nn.Linear(in_features=64,out_features=10)
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=x.view(x.size(0),-1)
        x=self.fc(x)
        output=self.out(x)
        return output
    
model=ConvNet()
print(model)

ConvNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1568, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
  )
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [15]:
import hiddenlayer as hl
model_img=hl.build_graph(model,torch.zeros([1,1,28,28]))
model_img.theme=hl.graph.THEMES['blue'].copy()
plt.show()

TypeError: 'torch._C.Node' object is not subscriptable

In [5]:
import shutil
from torchviz import make_dot
x=torch.randn(1,1,28,28).requires_grad_(True)
y=model(x)
model_img=make_dot(y,params=dict(list(model.named_parameters())+[('x',x)]))
model_img.format='png'
model_img.directory='tmp'
model_img.view()
#shutil.rmtree('tmp')

'tmp\\Digraph.gv.png'

In [20]:
import graphviz